# Tutorial: Training Data Generation for Projection Upsampling Network

## Overview
This notebook presents a comprehensive walkthrough of the training data generation process for 2D SIM reconstruction. The approach focuses on generating paired datasets consisting of noisy inputs and corresponding high-quality SIM reconstruction images derived from 3D raw microscope data.

## Objectives
- **Dataset Creation:** Develop pairs comprising noisy raw images and ground-truth images obtained with high signal-to-noise ratio (SNR).
- **Data Source:** Utilize the Microtubules dataset from BioSR for demonstration purposes. More details can be found at the [BioSR website](https://figshare.com/articles/dataset/BioSR/13264793).
- **Dataset Size:** For the demonstration, a small dataset with 5 patches per sample is generated. In practical scenarios, a larger dataset is recommended.
- **Tool Utilization:** Employ the CSBDeep library to streamline the data generation process. More information is available on the [CSBDeep documentation](https://csbdeep.bioimagecomputing.com/doc/).

## Steps
1. **Data Preparation:** Process the raw 3D noisy microscope data.
2. **Pair Generation:** Create paired samples, aligning noisy data with corresponding ground-truth SIM reconstruction images.
3. **Data Augmentation:** Apply augmentation techniques to enhance dataset variability and robustness. (saves on disk)
4. **Analysis and Validation:** Assess the quality of generated pairs using visualization and statistical metrics.

## Conclusion
This structured guide details the procedures for generating training data essential for developing projection upsampling networks, specifically optimized for 2D SIM reconstruction tasks.

In [11]:


import numpy as np
import matplotlib.pyplot as plt 
from pu_net.data_gen import create_folders_SR, create_patches_reduced_target, RawDataScaled
from csbdeep.utils import plot_some
import ipywidgets as widgets

import os
from pu_net.augmentation import flip_up_down, flip_left_right, transpose,  rotate_90, rotate_180, rotate_270



ModuleNotFoundError: No module named 'pu_net'